In [ ]:
#####################################################
# MODIFICARE QUESTA SEZIONE INSERENDO I PROPRI DATI #
#####################################################

 # identificativo del gruppo da verificare nel file excel
group_id = 31

 # nome della cartella Google Drive dove salvare i file
cartella_drive = "CorsoBioinformatica"

In [ ]:
# Installazione moduli aggiuntivi
!pip install biopython
!pip install git+https://github.com/Benjamin-Lee/CodonAdaptationIndex.git

  Cloning https://github.com/Benjamin-Lee/CodonAdaptationIndex.git to /tmp/pip-req-build-pacm1ujl
  Running command git clone --filter=blob:none --quiet https://github.com/Benjamin-Lee/CodonAdaptationIndex.git /tmp/pip-req-build-pacm1ujl
  Resolved https://github.com/Benjamin-Lee/CodonAdaptationIndex.git to commit b6e017a92c58829f6a5aec8c26a21262bc2a6610
  Preparing metadata (setup.py) ... done
  Created wheel for CAI: filename=CAI-1.0.5.dev3+gb6e017a.d20250315-py3-none-any.whl size=7850 sha256=4aff8fea7ea1993d3298c06f4502e67ed298d0f6df17f72261dec41cee8204d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-inl5f6t4/wheels/b3/07/05/fe24a5e008e49d2d8124e99d1bbe93c3d7115cdb4cfd1cb1d4
Successfully built CAI
  Attempting uninstall: CAI
    Found existing installation: CAI 1.0.5.dev3+gb6e017a.d20250315
    Uninstalling CAI-1.0.5.dev3+gb6e017a.d20250315:
      Successfully uninstalled CAI-1.0.5.dev3+gb6e017a.d20250315


In [ ]:
# Importazione moduli
import numpy as np
import pandas as pd
import os
import regex
import Bio
import Bio.SeqIO
from bisect import bisect_left
import pickle
from CAI import CAI
from Bio.Blast import NCBIWWW, NCBIXML

In [ ]:
# CARICAMENTO DELLA SEQUENZA FASTA
# scaricare il file
# siloe.dimes.unical.it/~fab/group_<group_id>_seq.fasta
!wget -q https://siloe.dimes.unical.it/~fab/MedicinaTD/group_{group_id}_seq.fasta

### Ricerca delle ORF sulla sequenza (parte del prof)
L'oggetto di classe match prende in input


*   string: la sequenza iniziale contenente tutti i potenziali geni
*   start: posizione iniziale
*   stop: posizione finale



Il metodo **group** restituisce la sottosequenza (singolo gene). Il metodo **span** restituisce una tupla con posizione iniziale e posizione finale

In [ ]:
# RICERCA DELLE ORF
# orf_iter restituisce un iteratore sulle orf
# le modalità d'uso sono equivalenti al find_iter di regex

class Match:
	def __init__(self, string, start, stop):
		self.string = string
		self.start_pos = start
		self.stop_pos = stop
		self._group0 = string[start:stop]
	def group(self):
		return self._group0
	def span(self):
		return (self.start_pos, self.stop_pos)
	def __repr__(self):
		return f"<Match object; span={self.span()}, match='{self.group()}'>"

La funzione `orf_iter `
1. crea una lista vuota con i codoni di stop
2. usa una stringa regex per trovare i codoni di stop nella sequenza e per ogni stop trovato appende aggiunge la posizione iniziale alla lista
3. usa una stringa regex per trovare i codoni di start, poi con un ciclo while cerca il primo stop ad una distanza di codoni multiplo di 3
4. dato che alla fine usa yield, restituisce un oggetto generatore che contiene i match, sul quale poi dovremo iterare con un ciclo

In [ ]:
def orf_iter(sequence):
	# Cerco tutti i codoni di stop e salvo il punto di partenza
	stops = []
	for stop in regex.finditer(r"TAA|TAG|TGA", sequence):
		stops.append(stop.span()[0])
	# Itero sugli start
	for start in regex.finditer(r"ATG", sequence):
		# cerco il primo stop con valore di distanza multiplo di 3
		pos_start = start.span()[0]
		pos_stop = bisect_left(stops, pos_start + 3)

		while (pos_stop < len(stops)) and ((stops[pos_stop] - pos_start) % 3 != 0):
			pos_stop = pos_stop + 1
		if pos_stop < len(stops):
			yield Match(sequence, pos_start, stops[pos_stop])

### Gene prediction

In questa parte
- carico la sequenza (è un file fasta e qui ho usato la funzione parse del modulo SeqIO di biopython per ottenere la sequenza).
- creo il df con i risultati
- calcolo anche le frequenze di CG in tutta la sequenza

In [ ]:
#caricamento sequenza
seq = next(Bio.SeqIO.parse(f"group_31_seq.fasta", "fasta")).seq
#setup df
results = pd.DataFrame(columns=["ORFpos","Coding","BLAST"])

# Calcolo frequenze CG in tutta la sequenza
exp_cpg = seq.count("C")/len(seq) * seq.count("G")/len(seq)

#### Definizioni funzioni per sensori di segnale
- tatabox: controlla se è presente un tatabox con una stringa regex
- inr: controlla se è presente un iniziatore con una stringa regex
- cpg: le isole cpg sono regioni specifiche del DNA ricche di nucleotidi CG (dove una C è seguita direttamente da una G). Sono definite da criteri specifici
  - localizzate a monte della sequenza codificante
  - lunghezza > 200 basi
  - contenuto GC > 0,5
  - rapporto CpG osservato/atteso > 0,6
- sequenza di Kozak: una sequenza consenso che circonda il codone di start (ATG) negli eucarioti

In [ ]:
def tatabox(seq):
    if regex.search(r"TATA[AT]A[AT]", str(seq)):
        return True
    return False

def inr(seq):
    if regex.search(r"[CT][CT]A[AGCT][AT][CT][CT]", str(seq)):
        return True
    return False

def cpg(seq, cpg_tot):
    #genero una sottosequenza a monte di quella coficante (3000 bp prima)
    seq_monte = str(seq[orf.span()[0]-3000:orf.span()[0]])
    #contenuto G e G
    GC_monte = len(regex.findall(r"[GC]", seq_monte))/3000
    #per prima cosa la frequenza deve essere superiore a 0.5
    if GC_monte > 0.5:
        #rapporto osservato/atteso
            obs_CG = seq_monte.count("CG")/3000
            #inoltre il rapporto osservato(nella orf)/atteso(in tutta la sequenza) deve essere maggiore di 0.6
            if obs_CG/cpg_tot > 0.6:
                #se è così aggiunto 1 punto
                return True
    return False

def kozak(seq):
    if regex.search(r"[AG]CCATGG", str(seq)):
        return True
    return False

#### Definizione di funzioni per sensori di contenuto
- in generale nelle regioni codificanti il contenuto di GC varia da 0,45 a 0,65
- CAI (codon adaptability index): nelle regioni codificanti c'è una preferenza per alcuni codoni rispetto ad altri a parità di amminoacido (codoni sinonimi)
  - Es. Serina → TCT, TCC, TCA, TCG, AGT, AGC
  - ho usato la funzione CAI() del pacchetto omonimo, che data la sequenza e un file con i pesi dei codoni, genera uno score

In [ ]:
def gc_content(seq):
    GC_content = len(regex.findall(r"[GC]", str(seq)))/3000
    if GC_content > 0.45 and GC_content < 0.65:
        return True
    return False

def codon_freq(seq):
    cai_dict = {"GCT": 1.000, "TTG": 0.600, "GCC": 0.900, "CTA": 0.400, "GCA": 0.700, "TTA": 0.300, "GCG": 0.600, "AAG": 1.000, "CGT": 1.000, "AAA": 0.800, "CGC": 0.900, "ATG": 1.000, "AGA": 0.700, "TTC": 1.000, "CGG": 0.600, "TTT": 0.800, "AGG": 0.500, "CCG": 1.000, "CGA": 0.400, "CCC": 0.900, "AAC": 1.000, "CCT": 0.800, "AAT": 0.800, "CCA": 0.700, "GAC": 1.000, "AGC": 1.000, "GAT": 0.800, "TCT": 0.900, "TGC": 1.000, "TCC": 0.800, "TGT": 0.800, "AGT": 0.700, "CAG": 1.000, "TCA": 0.600, "CAA": 0.700, "TCG": 0.500, "GAG": 1.000, "ACC": 1.000, "GAA": 0.800, "ACT": 0.900, "GGC": 1.000, "ACA": 0.700, "GGT": 0.900, "ACG": 0.600, "GGA": 0.700, "TGG": 1.000, "GGG": 0.600, "TAC": 1.000, "CAC": 1.000, "TAT": 0.800, "CAT": 0.800, "GTG": 1.000, "ATC": 1.000, "GTC": 0.900, "ATT": 0.900, "GTT": 0.800, "ATA": 0.500, "GTA": 0.600, "CTG": 1.000, "TAA": 1.000, "CTC": 0.800, "TGA": 0.900, "CTT": 0.700, "TAG": 0.800}
    cai_score=CAI(seq, weights=cai_dict)
    if cai_score > 0.8:
        return True
    return False

#### Funzione per verifica sequenza su BLAST
Qui uso la funzione qblast del modulo NCBIWWW che permette di cercare una sequenza su un database (refseq_rna). blastn è una variante di blast che confronta sequenze di nucleotidi con un database di nucleotidi.

La funzione restituisce un file XML che poi viene letto dalla funzione read del modulo NCBIXML, che usa un parser interno.

Il record che restituisce contiene un valore E-value che è una sorta di p-value (indica la confidence dell'allineamento). Se è sopra il valore soglia di 10^(-6) la funzione restituisce True

Ho usato try ed except in caso di eventuali errori durante l'allineamento

In [ ]:
def verifica_sequenza_blast(seq):
    try:
        result_handle = NCBIWWW.qblast(
            program="blastn",
            database="refseq_rna",
            sequence=seq
        )

        blast_records = NCBIXML.read(result_handle)

        # Controlla se ci sono match con E-value < 1e-6
        for alignment in blast_records.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < 1e-6:
                    return True
        return False
    except Exception as e:
        print(f"Errore durante l'esecuzione di BLAST: {e}")
        return False

#### Variabili
Qui ho raccolto le variabili di score e posizionali per modificarle più agevolmente

In [ ]:
#VARIABILI SCORE
score_tatabox = 3
score_inr = 2
score_cpg = 1
score_kozak = 2
score_gc = 1
score_codon = 1

#SCORE SOGLIA
soglia = 5

#VARIABILI POSIZIONALI
inizio_tatabox = -40
fine_tatabox = -20
inizio_inr = -10
fine_inr = 10
inizio_cpg = -3000
inizio_kozak = -10
fine_kozak = 10

#### Richiamo funzioni e assegnazione score
Itero su ogni orf nell'oggetto generatore creato da orf_iter sulla stringa contenente la sequenza.

Per prima cosa vengono eliminate le sequenze più piccole di 150bp, con continue l'iterazione corrente viene saltata.


Poi creo la variabile score e assegno lo score inziale di 0

A questo punto creo una sottosequenza che inizia 35 codoni prima dell'inizio dell'orf e finisce dopo 10 codoni (25 codoni prima dell'inizio dell'orf). Le variabili posizionali si possono modificare sopra.

Se c'è il tatabox (la funzione tatabox() restituisce V o F), lo score viene aumentato di 3 (modificabile sopra).

Viene fatta una cosa simile con tutti i sensori di segnale facendo attenizone a non andare out of bound con alcuni sensori, come l'inr. In quel caso viene aggiunto un controllo che in realtà dovrebbe esserci anche con altri sensori di segnale dato che potrebbe capitare un ATG a inizio sequenza

I sensori di contenuto sono propri della sequenza, io qui ho creato una sottosequenza in questo modo


```
subseq = str(seq[orf.span()[0]:orf.span()[1]])
```

ma si può usare anche il metodo `.group()` che restituisce direttamente la sequenza.

Infine, se lo score supera la soglia, viene aggiunto il valore True alla colonna "Coding" del df results tramite `results.loc[len(results)-1, 'Coding'] = True`.

Poi viene chiamata la funzione `verifica_sequenza_blast()` e se la sequenza è codificante viene aggiunto il valore true alla colonna "BLAST" del df results in modo simile

Infine vengono filtrate e stampate solo le orf codificanti e verificate su blast

In [ ]:
#inizio analisi
for orf in orf_iter(str(seq)):
  # FILTRAGGIO PRELIMINARE
    if orf.span()[1] - orf.span()[0] < 150:
        continue
    results.loc[len(results)] = [orf.span()[0], False, False]

    #PARTE CODING
    #inizializzo score
    score = 0
    #il tatabox si trova tra -35 e -25 prima dell'ATG iniziale
    subseq = str(seq[orf.span()[0]+inizio_tatabox:orf.span()[0]+fine_tatabox])
    if tatabox(subseq):
        score = score + score_tatabox
    subseq = str(seq[orf.span()[0]+inizio_inr:orf.span()[0]+fine_inr])
    #l'inr si trova tra -10 e +10 attorno all'ATG iniziale
    if inr(subseq):
        score = score + score_inr
    #solo se la sequenza inizia dopo 3000 basi
    if orf.span()[0] > 3000:
        subseq = str(seq[orf.span()[0]+inizio_cpg:orf.span()[0]])
        #controllo se nelle 3000 basi precedenti c'è un'alta frequenza di CpG
        if cpg(subseq, exp_cpg):
            score = score + score_cpg
    #il kozak si trova tra -10 e +10 attorno all'ATG iniziale
    subseq = str(seq[orf.span()[0]+inizio_kozak:orf.span()[0]+fine_kozak])
    if kozak(subseq):
        score = score + score_kozak
    #i sensori di contenuto si applicano alla sequenza da ATG a stop
    subseq = str(seq[orf.span()[0]:orf.span()[1]])
    if gc_content(subseq):
        score = score + score_gc
    subseq = str(seq[orf.span()[0]:orf.span()[1]])
    if codon_freq(subseq):
        score = score + score_codon
    if score >= soglia:
        results.loc[len(results)-1, 'Coding'] = True
        #PARTE VERIFICA BLAST
        print(f'Controllo se la sequenza in posizione {orf.span()} è verificata su BLAST')
        if verifica_sequenza_blast(subseq):
            print(f'La sequenza in posizione {orf.span()} è verificata su BLAST')
            results.loc[len(results)-1, 'BLAST'] = True

#filtra solo le ORF codificanti
cod = results[results['Coding'] == True]
print(cod)

#filtra solo le ORF codificanti verificate su BLAST
ver = results[results['BLAST'] == True]
print(ver)

Controllo se la sequenza in posizione (768638, 768791) è verificata su BLAST
Controllo se la sequenza in posizione (2143279, 2143573) è verificata su BLAST
La sequenza in posizione (2143279, 2143573) è verificata su BLAST
Controllo se la sequenza in posizione (2301612, 2301846) è verificata su BLAST
La sequenza in posizione (2301612, 2301846) è verificata su BLAST
Controllo se la sequenza in posizione (2357000, 2357276) è verificata su BLAST
La sequenza in posizione (2357000, 2357276) è verificata su BLAST
       ORFpos  Coding  BLAST
1363   768638    True  False
3751  2143279    True   True
4029  2301612    True   True
4158  2357000    True   True
       ORFpos  Coding  BLAST
3751  2143279    True   True
4029  2301612    True   True
4158  2357000    True   True


#### Salvataggio risultati su pickle
Qui viene scritto un file pickle con il df results

In [ ]:
# SALVATAGGIO DEI RISULTATI SU FILE
with open(f"group_{group_id}_results.pickle", "wb") as f:
	pickle.dump(results, f)